# Notebook para classificação de estrelas usando redes neurais com Keras
## Alexandre Suaide

Este notebook treina uma rede neural MLP usando **Keras/TensorFlow** para classificar tipos de estrelas a partir de suas propriedades físicas.


In [ ]:
# Se estiver no Google Colab, descomente a linha abaixo para instalar dependências:
# !pip install mdsdata tensorflow scikit-learn seaborn joblib --quiet

In [ ]:
# importa os módulos necessários
# dependendo da versão do TensoFlow, aparece um monte de warnings

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
# lê o dataset do arquivo CSV e coloca em um dataframe do Pandas

filename = "data-stars.csv" 

df = pd.read_csv(filename)
df

In [ ]:
# Faz o Diagrama H-R

lx = 'Temperature (K)'
ly = 'Luminosity (L/Lo)'
plt.figure(figsize=(6,5))
plt.scatter(df[lx], df[ly], s=20, alpha=0.6)
plt.yscale('log')
plt.gca().invert_xaxis()
plt.xlabel(lx)
plt.ylabel(ly)
plt.title('Diagrama H-R')
plt.show()

In [ ]:
# Determina quais features serão utilizados e o target para treinamento

FEATURES = ['Temperature (K)', 'Luminosity (L/Lo)', 'Radius (R/Ro)', 'Absolute magnitude (Mv)']
TARGET = 'Star category'

X = df[FEATURES].values
y_raw = df[TARGET].values

# Codifica os rótulos, já que o Target é uma variável de texto

le = LabelEncoder()
y = le.fit_transform(y_raw)
class_names = list(le.classes_)
print("Classes:", class_names)

# divisão do dataset em treino, teste e validação. 
# as variáveis abaixo determinam as frações

train_ratio = 0.50
test_ratio = 0.35
validation_ratio = 0.15

random_state = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, stratify = y, random_state = random_state)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), stratify = y_test, random_state = random_state) 

print(f"train = {len(X_train)} \ntest = {len(X_test)} \nval = {len(X_val)}")

# Normaliza as variáveis usando o scaler padrão

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
# Construção do modelo no Keras

model = keras.Sequential([
    layers.Input(shape=(len(FEATURES),)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# treinamento da rede
# verbose = 0, 1 ou 2 indica a quantidade de print

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    verbose=1
)

In [ ]:
# faz figura da perda e da acurácia para cada época

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='treino')
plt.plot(history.history['val_loss'], label='validação')
plt.title('Perda')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='treino')
plt.plot(history.history['val_accuracy'], label='validação')
plt.title('Acurácia')
plt.legend()
plt.show()

In [ ]:
# Avaliar o dataset de teste
# obtém a matriz de confusão

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.4f}")

y_pred = np.argmax(model.predict(X_test), axis=1)

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap='Blues')
plt.xlabel('Predição')
plt.ylabel('Verdade')
plt.title('Matriz de confusão')
plt.show()

In [ ]:
# Fazer predições e printar o resultado

n = len(X_test)
samples = X_test[:n]
pred_labels = le.inverse_transform(np.argmax(model.predict(samples), axis=1))
true_labels = le.inverse_transform(y_test[:n])

for i in range(n):
    print(f"Pred: {pred_labels[i]}  \t Verd: {true_labels[i]}  \t {pred_labels[i] == true_labels[i]}")

In [ ]:
# salva o modelo para uso futuro em novos dados. Dai não precisa treinar a rede novamente

model.save("star_classifier_keras.keras")
print("Modelo gravado como star_classifier_keras.keras")